In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from quickdraw import QuickDrawDataGroup

from bwb.distributions import DistributionDraw

In [2]:
data = np.load(r"..\data\face.npy")
data.shape

(161666, 784)

Hacer una lista de caritas

In [3]:
rng = np.random.default_rng(42)
faces: list[DistributionDraw] = list()
n_faces = len(data)

for i in range(n_faces):
    arr = 255 - data[i, :].reshape(28, 28)
    dd_arr = DistributionDraw.fromarray(arr, seed=rng)
    faces.append(dd_arr)

faces[:10]

In [4]:
faces[0].image

Muestrear con respecto a una carita (la primera para simplificar)

In [5]:
n = 3
x = faces[0].rvs(size=n)
x = [tuple(x[i]) for i in range(n)]
x

[(21, 15), (11, 10), (22, 18)]

Defino
\begin{equation}
\tilde\Lambda_n(m) = \prod_{i=1}^{n} f_m(x_i)
\end{equation}

Pues así
\begin{equation}
\Lambda_n(m) = \frac{\tilde\Lambda_n(m)}{\int_{\mathcal{M}} \tilde\Lambda_n(\tilde m) \Pi(\text{d} \tilde m) }
\end{equation}

In [6]:
def lambda_n_tilde(mu: DistributionDraw, data: list[tuple[int, int]]):
    evaluations = []
    for i in range(len(data)):
        evaluations.append(mu.matrix[data[i]])
    return np.prod(evaluations)
lambda_n_tilde(mu=faces[0], data=x)

6.001854530919468e-08

In [7]:
N = 10000
mu = []
# Paso 1. Empezar con una distribución inicial mu^(0) \in M
current_face = int(rng.choice(n_faces))
print(f"{current_face = }")
mu.append(faces[current_face])  # mu^(0)

# Paso 2.
for i in range(N):
    # Paso a. muestrear u ~ U[0, 1]
    u = rng.uniform(low=0, high=1)
    # Paso b. muestrear mu^star de acuerdo a R partiendo desde mu^(i)
    possible_faces = list(range(n_faces))
    possible_faces.remove(current_face)
    next_face = int(rng.choice(possible_faces))
    mu_star = faces[next_face]
    # Paso c.
    # Empecemos calculando R
    R_mu_star_mu_i = 1 / (n_faces - 1)
    R_mu_i_mu_star = 1 / (n_faces - 1)
    lambda_mu_star_lambda_mu_i = lambda_n_tilde(mu_star, data=x) / (lambda_n_tilde(mu[i], data=x) + 1e-16)
    A_mu_i_mu_star = min(1, (lambda_n_tilde(mu_star, data=x) * R_mu_star_mu_i)
                            / (lambda_n_tilde(mu[i], data=x) * R_mu_i_mu_star + 1e-16))
    if u < A_mu_i_mu_star:
        mu.append(mu_star)

        print(f"{i = }")
        print(f"{lambda_mu_star_lambda_mu_i = }")
        print(f"{A_mu_i_mu_star = }")
        print(f"{u = }")
    else:
        mu.append(mu[i])

current_face = 13894
i = 1
lambda_mu_star_lambda_mu_i = 1202628740.1060722
A_mu_i_mu_star = 1
u = 0.9756223516367559
i = 36
lambda_mu_star_lambda_mu_i = 0.6078325446877919
A_mu_i_mu_star = 0.6077508474610039
u = 0.13979699812765745
i = 86
lambda_mu_star_lambda_mu_i = 1.159852699516265
A_mu_i_mu_star = 1
u = 0.34686980453483707
i = 90
lambda_mu_star_lambda_mu_i = 0.8966792153158372
A_mu_i_mu_star = 0.8965082731291945
u = 0.08272999992243857
i = 99
lambda_mu_star_lambda_mu_i = 1.9680109457353483
A_mu_i_mu_star = 1
u = 0.14024908899861077
i = 163
lambda_mu_star_lambda_mu_i = 0.583649177113312
A_mu_i_mu_star = 0.5835861198245356
u = 0.3741840434071827
i = 168
lambda_mu_star_lambda_mu_i = 1.7858465527206764
A_mu_i_mu_star = 1
u = 0.8165687634239104
i = 174
lambda_mu_star_lambda_mu_i = 1.007279611222232
A_mu_i_mu_star = 1
u = 0.16553172273527816
i = 193
lambda_mu_star_lambda_mu_i = 0.1951222318173529
A_mu_i_mu_star = 0.19510215262041755
u = 0.0991131415798755
i = 197
lambda_mu_star_lambda_mu

In [8]:
mu[0].image

In [9]:
mu[:10]

In [10]:
mu[-1].image